In [ ]:
import pandas as pd
from utils import aggregate_data, join_col_descriptions, categorize_school_level, PostGresLoad

%load_ext autoreload
%autoreload 2

In [ ]:
cd ~/Desktop/pleiades_archive/discriminology/

In [ ]:
pg = PostGresLoad()
newark = pd.read_csv('newark_schools.csv')
pg.df_to_rds(newark, 'newark')

In [ ]:
prev_year_data = pd.read_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_schools.csv', index_col=0).rename(columns={'index': 'COMBOKEY'})
prev_year_data.head()

In [ ]:
prev_year_data['YEAR'].value_counts()

Read field mapping and data types from [dictionary](https://docs.google.com/spreadsheets/d/1mO7Tnbn9hwHcJ0hj8gBBp5MhHGDqmgGw8X3-b-s6SKs/edit#gid=0)

In [ ]:
field_mappings = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/field_mapping.xlsx')

In [ ]:
field_mappings.head()

In [ ]:
field_mappings['col_superset'] = field_mappings['15_16_field'].combine_first(field_mappings['11_12_field'])
type_map = pd.Series(field_mappings.type.values, index=field_mappings.col_superset).to_dict()
numeric_type_map = {k:v for k,v in type_map.items() if v !='str'}

In [ ]:
newark = pd.read_csv('/Users/cave/Desktop/pleiades_archive/discriminology/newark_schools.csv', dtype={'COMBOKEY':str})
newark.head()

### Extract methodology fields for each year

In [ ]:
fields_1112 = field_mappings['11_12_field'].dropna().values
fields_1314 = field_mappings['13_14_field'].dropna().values
fields_1516 = field_mappings['15_16_field'].dropna().values
fields_1718 = field_mappings['17_18_field'].dropna().values

In [ ]:
cd ~/Desktop/pleiades_archive/discriminology/2017_18/Data/SCH/CRDC/CSV

### Loop through files in 2017-18 OCR dump and pull out variables of interest.

In [ ]:
df_17_18 = pd.read_csv('Enrollment.csv',  dtype={'COMBOKEY':str})
files_17_18 = ['Gifted and Talented.csv', 'Suspensions.csv', 'Advanced Placement.csv', 'School Characteristics.csv',
              'Referrals and Arrests.csv', 'Single-sex Classes.csv', 'School Support.csv'
              ]

join_keys = ['LEA_NAME', 'SCH_NAME', 'SCHID', 'LEAID', 'JJ', 'LEA_STATE_NAME']
for file in files_17_18:
    temp = pd.read_csv(file, dtype={'COMBOKEY':str})
    col_subset = [c for c in temp.columns if c not in join_keys]
    df_17_18 = pd.merge(df_17_18, temp[col_subset], on='COMBOKEY')


In [ ]:
df_17_18 = df_17_18[fields_1516]
df_17_18.head()

In [ ]:
df_17_18.shape

### Check how many Newark schools are missing from the 17-18 data

In [ ]:
len(newark)
merged = pd.merge(newark, df_17_18, on='COMBOKEY', how='left')
merged['LEAID'].isna().sum()

Generate SQL create statement for school table in AWS

In [ ]:
create_school_table = [print(f'"{k}" double precision,') for k,v in numeric_type_map.items()]
create_school_table

In [ ]:
universal_types = {'SCH_ZIP ': str
                   ,'SCHID': str
                   ,'COMBOKEY': str
                   ,'LEAID': str
                  }

### Clean up data types

In [ ]:
df_17_18['LEAID'] = df_17_18['LEAID'].astype(str).str.zfill(7)
df_17_18['SCHID'] = df_17_18['SCHID'].astype(str).str.zfill(5)
df_17_18['COMBOKEY'] = df_17_18['LEAID'] + df_17_18['SCHID']

In [ ]:
df_17_18.head()

### Replace negative values with zeroes.

In [ ]:
num = df_17_18._get_numeric_data()
num[num < 0] = 0
df_17_18 = df_17_18.replace('-5', 'No').replace('-9', 'No')
df_1718_final = df_17_18
df_1718_final['YEAR'] = '2017-18'
df_1718_final.head()

In [ ]:
df_1718_final = df_1718_final.astype(numeric_type_map)
df_1718_final.head()

### Concatenate 2017-18 data to previous years

In [ ]:
all_years = pd.concat([df_1718_final, prev_year_data], axis=0)

In [ ]:
all_years['YEAR'].value_counts()

In [ ]:
all_years['YEAR'].value_counts()

In [ ]:
full_table = all_years

In [ ]:
full_table = full_table.astype(numeric_type_map)
# full_table.sort_index(inplace=True)
full_table.reset_index(inplace=True)
full_table['SCHID'] = full_table['SCHID'].astype(str).str.zfill(5)

Impute missing district state names from other years 

In [ ]:
# Fill in LEA STATE with non-null values in LEAID group
full_table['LEA_STATE'] = full_table.groupby(['LEAID'])['LEA_STATE'].fillna(method='ffill').fillna(method='bfill')
full_table['LEA_STATE_NAME'] = full_table.groupby(['LEAID'])['LEA_STATE_NAME'].fillna(method='ffill').fillna(method='bfill')
full_table['LEA_NAME'] = full_table.groupby(['LEAID'])['LEA_NAME'].fillna(method='ffill').fillna(method='bfill')

In [ ]:
full_table['SCH_NAME'] = full_table.groupby(['SCHID'])['SCH_NAME'].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_ZIP '] = full_table.groupby(['SCHID'])['SCH_ZIP '].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_CITY'] = full_table.groupby(['SCHID'])['SCH_CITY'].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_ADDRESS'] = full_table.groupby(['SCHID'])['SCH_ADDRESS'].fillna(method='ffill').fillna(method='bfill')

Capitalize the first letter in each district name, school name, address.

In [ ]:
full_table['LEA_NAME'] = full_table['LEA_NAME'].str.title()
full_table['SCH_NAME'] = full_table['SCH_NAME'].str.title()
full_table['SCH_ADDRESS'] = full_table['SCH_ADDRESS'].str.title()
full_table['SCH_CITY'] = full_table['SCH_CITY'].str.title()

Categorize each school as elementary, middle, high, or other using grade enrollment data.

In [ ]:
full_table['SCH_LEVEL'] = full_table.apply(lambda row: categorize_school_level(row['SCH_GRADE_KG'], row['SCH_GRADE_G01'],
                            row['SCH_GRADE_G02'], row['SCH_GRADE_G03'], row['SCH_GRADE_G04'], row['SCH_GRADE_G05'],
                            row['SCH_GRADE_G06'], row['SCH_GRADE_G07'], row['SCH_GRADE_G08'], row['SCH_GRADE_G09'],
                            row['SCH_GRADE_G10'], row['SCH_GRADE_G11'], row['SCH_GRADE_G12']), axis=1
                            )

In [ ]:
full_table.to_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_schools.csv', index=False)

In [ ]:
full_table = pd.read_csv

In [ ]:
full_table.shape

In [ ]:
# full_table.set_index('COMBOKEY', inplace=True)

In [ ]:
prev_year_data.shape

### Group by districts and sum over fields

In [ ]:
district = full_table.copy()

In [ ]:
# district.columns = district.columns.droplevel(1)
district = district.astype(numeric_type_map)

In [ ]:
district['TOTAL_SCHOOLS'] = 1.0
district['TOTAL_ENROLLMENT'] = district['TOT_ENR_F'] + district['TOT_ENR_M']

Custom aggregation dictionary - for string values, take the first non-null value.  For numeric fields, take the sum

In [ ]:
agg_cols = ['LEA_NAME', 'LEA_STATE_NAME', 'TOTAL_SCHOOLS', 'TOTAL_ENROLLMENT',
'SCH_DISCWODIS_MULTOOS_HI_M',
 'SCH_DISCWDIS_ARR_IDEA_AM_M',
 'SCH_GTENR_IDEA_F',
 'SCH_DISCWODIS_SINGOOS_HP_F',
 'SCH_FTECOUNSELORS',
 'SCH_DISCWODIS_ARR_WH_M',
 'TOT_DISCWDIS_ARR_IDEA_M',
 'SCH_GTENR_TR_M',
 'SCH_DISCWODIS_MULTOOS_AS_M',
 'SCH_APENR_AM_F',
 'SCH_DISCWODIS_REF_BL_F',
 'SCH_DISCWODIS_ARR_AM_F',
 'TOT_DISCWODIS_REF_M',
 'SCH_IDEAENR_HI_M',
 'SCH_FTESERVICES_PSY',
 'SCH_DISCWDIS_MULTOOS_IDEA_BL_F',
 'TOT_APENR_F',
 'SCH_DISCWODIS_ARR_WH_F',
 'SCH_IDEAENR_HP_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_TR_F',
 'SCH_DISCWDIS_REF_IDEA_WH_F',
 'SCH_DISCWODIS_MULTOOS_WH_M',
 'SCH_FTESECURITY_GUA',
 'SCH_DISCWODIS_SINGOOS_TR_M',
 'SCH_DISCWDIS_REF_IDEA_WH_M',
 'SCH_DISCWODIS_ARR_TR_M',
 'SCH_DISCWDIS_ARR_IDEA_BL_F',
 'SCH_DISCWDIS_ARR_IDEA_TR_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_BL_M',
 'SCH_DISCWODIS_ARR_AM_M',
 'SCH_DISCWDIS_ARR_IDEA_AS_M',
 'SCH_DISCWDIS_REF_IDEA_BL_M',
 'SCH_GTENR_IDEA_M',
 'TOT_DISCWODIS_MULTOOS_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_HI_F',
 'TOT_DISCWODIS_SINGOOS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AM_F',
 'SCH_DISCWODIS_MULTOOS_TR_F',
 'TOT_DISCWODIS_SINGOOS_M',
 'SCH_ENR_HI_M',
 'SCH_APENR_HI_F',
 'SCH_IDEAENR_HI_F',
 'SCH_DISCWDIS_REF_IDEA_BL_F',
 'SCH_DISCWODIS_MULTOOS_AM_F',
 'TOT_DISCWDIS_MULTOOS_IDEA_M',
 'SCH_DISCWDIS_REF_IDEA_AS_M',
 'SCH_GTENR_HI_M',
 'SCH_DISCWODIS_REF_HI_M',
 'SCH_FTESERVICES_SOC',
 'SCH_DISCWDIS_SINGOOS_IDEA_AM_F',
 'SCH_DISCWODIS_REF_TR_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_HP_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_BL_M',
 'TOT_ENR_F',
 'SCH_GTENR_AS_F',
 'SCH_GTENR_HP_F',
 'SCH_ENR_BL_F',
 'SCH_DISCWODIS_SINGOOS_BL_M',
 'SCH_DISCWODIS_SINGOOS_TR_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_WH_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AS_F',
 'SCH_DISCWODIS_SINGOOS_HP_M',
 'SCH_GTENR_WH_M',
 'SCH_DISCWODIS_REF_AM_F',
 'SCH_DISCWODIS_SINGOOS_AS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_HI_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HI_M',
 'SCH_DISCWODIS_SINGOOS_HI_M',
 'SCH_APENR_TR_M',
 'SCH_DISCWODIS_REF_HI_F',
 'SCH_DISCWODIS_ARR_BL_F',
 'SCH_ENR_TR_F',
 'SCH_DISCWODIS_SINGOOS_BL_F',
 'SCH_DISCWODIS_ARR_HP_F',
 'SCH_APENR_AS_M',
 'SCH_GTENR_LEP_M',
 'SCH_DISCWDIS_REF_IDEA_HI_F',
 'SCH_DISCWODIS_MULTOOS_BL_F',
 'SCH_DISCWODIS_REF_HP_M',
 'SCH_APENR_BL_F',
 'SCH_IDEAENR_AS_M',
 'TOT_DISCWODIS_ARR_M',
 'SCH_DISCWODIS_SINGOOS_HI_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_WH_F',
 'SCH_IDEAENR_AM_F',
 'TOT_APENR_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_TR_M',
 'SCH_DISCWODIS_REF_TR_F',
 'SCH_DISCWDIS_REF_IDEA_AM_M',
 'SCH_GTENR_BL_F',
 'SCH_IDEAENR_TR_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AM_M',
 'SCH_GTENR_HI_F',
 'SCH_DISCWODIS_ARR_AS_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_TR_F',
 'TOT_DISCWODIS_MULTOOS_F',
 'SCH_DISCWODIS_REF_AS_M',
 'SCH_DISCWDIS_ARR_IDEA_WH_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_BL_F',
 'TOT_DISCWDIS_SINGOOS_IDEA_M',
 'SCH_APENR_HI_M',
 'SCH_DISCWODIS_SINGOOS_WH_F',
 'SCH_DISCWDIS_ARR_IDEA_HP_M',
 'SCH_IDEAENR_AM_M',
 'SCH_APENR_AM_M',
 'SCH_GTENR_TR_F',
 'TOT_GTENR_F',
 'SCH_DISCWODIS_SINGOOS_AM_F',
 'SCH_APENR_WH_M',
 'SCH_DISCWODIS_REF_AM_M',
 'TOT_DISCWODIS_REF_F',
 'SCH_IDEAENR_BL_F',
 'SCH_DISCWODIS_SINGOOS_AS_M',
 'TOT_DISCWDIS_SINGOOS_IDEA_F',
 'SCH_DISCWODIS_ARR_HI_M',
 'SCH_DISCWDIS_ARR_IDEA_HP_F',
 'SCH_DISCWDIS_REF_IDEA_HI_M',
 'SCH_ENR_TR_M',
 'SCH_GTENR_BL_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_AM_M',
 'SCH_DISCWODIS_ARR_HI_F',
 'SCH_IDEAENR_AS_F',
 'SCH_DISCWDIS_ARR_IDEA_HI_F',
 'SCH_APENR_HP_F',
 'TOT_IDEAENR_M',
 'SCH_GTENR_AM_M',
 'SCH_ENR_BL_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_AS_M',
 'SCH_ENR_AM_M',
 'SCH_DISCWODIS_MULTOOS_WH_F',
 'SCH_ENR_HP_M',
 'SCH_IDEAENR_BL_M',
 'SCH_IDEAENR_TR_M',
 'SCH_APENR_WH_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_AS_F',
 'SCH_ENR_AM_F',
 'SCH_DISCWDIS_REF_IDEA_TR_M',
 'SCH_DISCWODIS_REF_AS_F',
 'SCH_DISCWDIS_ARR_IDEA_AM_F',
 'SCH_ENR_WH_M',
 'SCH_DISCWODIS_REF_BL_M',
 'SCH_DISCWODIS_MULTOOS_AM_M',
 'SCH_DISCWDIS_ARR_IDEA_HI_M',
 'SCH_DISCWODIS_REF_HP_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_AS_M',
 'SCH_DISCWODIS_ARR_AS_M',
 'SCH_IDEAENR_WH_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_HP_F',
 'TOT_DISCWDIS_REF_IDEA_M',
 'SCH_FTESECURITY_LEO',
 'SCH_DISCWODIS_MULTOOS_TR_M',
 'SCH_IDEAENR_HP_F',
 'SCH_DISCWODIS_MULTOOS_HI_F',
 'SCH_DISCWDIS_ARR_IDEA_WH_F',
 'SCH_GTENR_HP_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_TR_M',
 'SCH_DISCWDIS_ARR_IDEA_AS_F',
 'SCH_DISCWODIS_REF_WH_M',
 'SCH_DISCWODIS_ARR_BL_M',
 'TOT_IDEAENR_F',
 'TOT_DISCWDIS_ARR_IDEA_F',
 'SCH_DISCWODIS_MULTOOS_AS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_WH_M',
 'SCH_APENR_TR_F',
 'SCH_ENR_WH_F',
 'SCH_DISCWODIS_REF_WH_F',
 'SCH_APENR_HP_M',
 'SCH_DISCWODIS_MULTOOS_BL_M',
 'SCH_GTENR_AM_F',
 'TOT_DISCWDIS_MULTOOS_IDEA_F',
 'SCH_DISCWDIS_REF_IDEA_HP_M',
 'SCH_ENR_HI_F',
 'SCH_DISCWDIS_REF_IDEA_HP_F',
 'TOT_ENR_M',
 'SCH_ENR_AS_F',
 'SCH_DISCWDIS_REF_IDEA_AM_F',
 'SCH_DISCWODIS_ARR_HP_M',
 'SCH_APENR_AS_F',
 'TOT_DISCWODIS_ARR_F',
 'SCH_ENR_AS_M',
 'SCH_GTENR_WH_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HP_F',
 'SCH_GTENR_LEP_F',
 'SCH_IDEAENR_WH_F',
 'SCH_DISCWODIS_SINGOOS_AM_M',
 'SCH_DISCWODIS_SINGOOS_WH_M',
 'SCH_DISCWDIS_ARR_IDEA_TR_F',
 'SCH_ENR_HP_F',
 'SCH_DISCWODIS_MULTOOS_HP_M',
 'SCH_DISCWDIS_REF_IDEA_TR_F',
 'SCH_GTENR_AS_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_WH_M',
 'TOT_GTENR_M',
 'SCH_DISCWODIS_ARR_TR_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HP_M',
 'SCH_DISCWODIS_MULTOOS_HP_F',
 'SCH_DISCWDIS_ARR_IDEA_BL_M',
 'SCH_DISCWDIS_REF_IDEA_AS_F',
 'SCH_APENR_BL_M',
 'TOT_DISCWDIS_REF_IDEA_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_HI_M']

In [ ]:
agg_dict = {k:sum if k not in {'LEA_NAME', 'LEA_STATE_NAME'} else 'first' for k in agg_cols}

In [ ]:
assert(len(agg_dict) == len(agg_cols))

In [ ]:
grouped_by_distyr = district.groupby(['LEAID', 'YEAR'])[agg_cols].agg(agg_dict)
grouped_by_distyr = grouped_by_distyr[agg_cols]
grouped_by_distyr.reset_index(inplace=True)
grouped_by_distyr.sort_values(by=['LEAID', 'YEAR'], inplace=True)

Read geodata from file and join to district aggregation.

In [ ]:
sch_geo_1516 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2015:16/Schools_EDGE_GEOCODE_PUBLICSCH_1516.xlsx', dtype={'NCESSCH': str})
dist_geo_1516 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2015:16/Districts_EDGE_GEOCODE_PUBLICLEA_1516.xlsx', dtype={'LEAID': str})


In [ ]:
sch_geo_1516['NCESSCH'] = sch_geo_1516['NCESSCH'].str.zfill(12)
dist_geo_1516['LEAID'] = dist_geo_1516['LEAID'].str.zfill(7)
sch_geo_1516.NCESSCH.nunique()

In [ ]:
sch_geo_1819 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2018:19/Schools_EDGE_GEOCODE_PUBLICSCH_1819.xlsx', dtype={'NCESSCH': str})
dist_geo_1819 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2018:19/Districts_EDGE_GEOCODE_PUBLICLEA_1819.xlsx', dtype={'LEAID': str})

In [ ]:
sch_geo_1819['NCESSCH'] = sch_geo_1819['NCESSCH'].str.zfill(12)
dist_geo_1819['LEAID'] = dist_geo_1819['LEAID'].str.zfill(7)
sch_geo_1819.NCESSCH.nunique()

In [ ]:
geo_data = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/LEA Profile Info.xlsx', dtype={'LEAID': str})
geo_data['LEAID'] = geo_data['LEAID'].str.zfill(7)

In [ ]:
len(geo_data)

In [ ]:
len(set(grouped_by_distyr['LEAID'].values))

In [ ]:
len(set(geo_data['LEAID'].values).intersection(set(grouped_by_distyr['LEAID'].values)))

In [ ]:
district_geo = geo_data[['LEAID', 'LEA_ADDRESS', 'LEA_CITY', 'LEA_ZIP',
       'CJJ', 'LEA_ENR', 'LEA_SCHOOLS', 'Latitude', 'Longitude']]

In [ ]:
district_geo.head()

In [ ]:
district_group_w_geo = pd.merge(grouped_by_distyr, district_geo, left_on='LEAID', right_on='LEAID', how='left')

In [ ]:
district_group_w_geo.head()

In [ ]:
text_cols = ['LEAID', 'LEA_NAME', "YEAR", "LEA_STATE", "LEA_STATE_NAME",
    "LEAID", "LEA_NAME", 'LEA_ADDRESS', 'LEA_CITY', 'CJJ',
    ]

In [ ]:
district_create = [print(f'"{k}" double precision,') if k not in text_cols else print(f'"{k}" text,') for k in district_group_w_geo.columns]

In [ ]:
dist_type_map = {k:v for k,v in type_map.items() if k in district_group_w_geo.columns}

In [ ]:
district_group_w_geo = district_group_w_geo.astype(dist_type_map)

In [ ]:
df = pd.read_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_districts.csv')

In [ ]:
df.shape

In [ ]:
district_group_w_geo.to_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_districts.csv', index=False)

### Code Sandbox - everything below only needs to be run once

In [ ]:
# descriptions = pd.concat(desc_frames_1112, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1112]
# descriptions.columns = ['2011_12_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2011_12_field_descriptions.csv')

# descriptions = pd.concat(desc_frames_1314, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1314]
# descriptions.columns = ['2013_14_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2013_14_field_descriptions.csv')